In [1]:
using ParallelTemperingMonteCarlo#feature/ensemble_variables



In [2]:
n_atoms = 55
ti = 750.
tf = 1200.
n_traj = 20
temp = TempGrid{n_traj}(ti,tf) 
mc_cycles = 1000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)
n = 8.482
m = 4.692
evtohartree = 0.0367493
nmtobohr = 18.8973
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]
 copperconstant = 0.36258*nmtobohr
 AtoBohr = 1.8897259886
 pos_cu13 = copperconstant*ico_13*1.5
 bc_cu13 = SphericalBC(radius=8*AtoBohr)
 start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-5.03606052999e-8, -4.52217680244e-8, -3.3916326018300003e-8], [-7.5150356316912e-6, -1.4388744371399999e-8, 6.736624211084434], [1.861953810320738, -5.73051487387157, 3.012712005572341], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.874670517929186, 3.541652881947096, 3.0127044987588487], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.025416511525561, 2.26108840122e-8, 3.0127165935262603], [-1.8619538082652034, -5.73051493450985, -3.012712020988853], [-6.025416612246772, -1.1202665260590001e-7, -3.0127166603311455], [-1.8619539110419487, 5.730514784455802, -3.0127120723772256], [4.87467035451416, 3.5416529415576075, -3.0127045974245252], [4.874670417207975, -3.541652971362864, -3.012704565563734], [7.414314421091401e-6, -7.502702422230001e-8, -6.736624278917086]], SphericalBC{Float64}(228.54811596737582))

In [3]:
dist2_mat = get_distance2_mat(start_config)

vars = set_variables(start_config,dist2_mat,pot)

EmbeddedAtomVariables([1.1280632578575638e-6 0.0015564309631457595; 4.065436622202774e-7 0.0007007747303521784; … ; 4.065436622550802e-7 0.000700774730365422; 4.0654366201822755e-7 0.000700774730149579], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0])

In [7]:
mutable struct teststruct{ptype}
    vars::ptype
    number::Int
end
testy = teststruct{typeof(vars)}(vars,6)

teststruct{EmbeddedAtomVariables}(EmbeddedAtomVariables([1.1280632578575638e-6 0.0015564309631457595; 4.065436622202774e-7 0.0007007747303521784; … ; 4.065436622550802e-7 0.000700774730365422; 4.0654366201822755e-7 0.000700774730149579], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0]), 6)

In [8]:
function unpacktype(x::teststruct{p}) where p
    println(p)
end
unpacktype(testy)

EmbeddedAtomVariables


In [4]:
@code_warntype atom_displacement([1.,1.,1.], 2.,bc_cu13)

MethodInstance for ParallelTemperingMonteCarlo

.MCMoves.atom_displacement(::Vector{Float64}, ::Float64, ::SphericalBC{Float64})
  from 

atom_displacement(pos, max_displacement, bc::SphericalBC) in ParallelTemperingMonteCarlo.MCMoves at /home/grayseff/.julia/packages/ParallelTemperingMonteCarlo/aMCBK/src/MCMoves.jl:29
Arguments
  #self#::Core.Const(ParallelTemperingMonteCarlo.MCMoves.atom_displacement)
  pos::Vector{Float64}
  max_displacement::Float64
  bc::SphericalBC{Float64}
Locals
  count::Int64
  trial_pos::StaticArraysCore.SVector{3, Float64}
  delta_move::StaticArraysCore.SVector{3, Float64}
Body::StaticArraysCore.SVector{3, Float64}


1 ─ %1  = ParallelTemperingMonteCarlo

.MCMoves.rand()

::Float64
│   %2  = (%1 - 0.5)::Float64
│   %3  = (%2 * max_displacement

)::Float64
│   %4  = ParallelTemperingMonteCarlo.MCMoves.rand()::Float64
│   %5  = (%4 - 0.5)::Float64
│   %6  = (%5 * max_displacement)::Float64
│   %7  = ParallelTemperingMonteCarlo.MCMoves.rand()::Float64
│   %8  = (%7 - 0.5)::Float64
│   %9  = (%8 * max_displacement)::Float64
│         (delta_move = ParallelTemperingMonteCarlo.MCMoves.SVector(%3, %6, %9))
│         (trial_pos = pos + delta_move)
└──       (count = 0)
2 ┄ %13 = ParallelTemperingMonteCarlo.MCMoves.check_boundary(bc, trial_pos)::Bool
└──       goto #6 if not %13
3 ─       (count = count + 1)
│   %16 = ParallelTemperingMonteCarlo.MCMoves.rand()::Float64
│   %17 = (%16 - 0.5)::Float64
│   %18 = (%17 * max_displacement)::Float64
│   %19 = ParallelTemperingMonteCarlo.MCMoves.rand()::Float64
│   %20 = (%19 - 0.5)::Float64
│   %21 = (%20 * max_displacement)::Float64
│   %22 = ParallelTemperingMonteCarlo.MCMoves.rand()::Float64
│   %23 = (%22 - 0.5)::Float64
│   %24 = (%23 * max_displacement)::Float64
│         (delta_move =

"Error: too many moves out of binding sphere")
└──       

Core.Const(:(goto %31))
5 ┄       goto #2
6 ─       return trial_pos



In [10]:
@time atom_displacement([1.,1.,1.], 2.,bc_cu13)

  0.000013 seconds (2 allocations: 112 bytes)


3-element StaticArraysCore.SVector{3, Float64} with indices SOneTo(3):
 0.18141938575960892
 1.748288350537059
 0.7696043606598988

In [ ]:
abstract type testtype end 
struct testtype1{T,N} where T 
    index::T
end
struct testtype2{T,N} where T
    index::T 
end


In [29]:
function printyprint(::SphericalBC)
    println("SBC")
end
function printyprint(::PeriodicBC)
    println("notSBC")
end

function returntypes(state::MCState{T,N,BC}) where {T,N,BC}
    printyprint(BC)
end

returntypes(state)

MethodError: MethodError: no method matching printyprint(::Type{SphericalBC{Float64}})
Closest candidates are:
  printyprint(!Matched::SphericalBC) at ~/Code/ParallelTemperingMonteCarlo.jl/scripts/testingspace.ipynb:1
  printyprint(!Matched::PeriodicBC) at ~/Code/ParallelTemperingMonteCarlo.jl/scripts/testingspace.ipynb:4

In [9]:
using StaticArrays

In [25]:
function testatommove(mc_state::MCState{T,N,BC}) where {T,N,BC<:PeriodicBC}
    #mc_state.ensemble_variables.trial_move = testmove(BC,mc_state.config.pos[mc_state.ensemble_variables.index],mc_state.max_displ[1],mc_state.config.bc)
    println("PBC")
end

function testatommove(mc_state::MCState{T,N,BC}) where {T,N,BC<:SphericalBC}
    #mc_state.ensemble_variables.trial_move = testmove(BC,mc_state.config.pos[mc_state.ensemble_variables.index],mc_state.max_displ[1],mc_state.config.bc)
    return 
end

testatommove (generic function with 2 methods)

In [26]:
testatommove(state)

In [27]:
using Profile

In [28]:
@profview for i in 1:1000 testatommove(state) end

┌ Warning: There were no samples collected.
│ Run your program longer (perhaps by running it multiple times),
│ or adjust the delay between samples with `Profile.init()`.
└ @ Profile /cache/build/default-amdci4-3/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Profile/src/Profile.jl:1235
